# FOP Demonstator Sheet



## Import the necessary library

In [1]:
import pandas as pd
import os
import re

In [2]:
month = "October_2023"
folder_path = "C://Users//User//Desktop//UM//FOP_Demo//FOPDemoExcelScript//" + month + "//"
file_prefix = "WIX1002_AttendanceSession_"

In [3]:
info = {
    'Course': 'WIX1002 FUNDAMENTALS OF PROGRAMMING',
    'Group': '2023/2024_SEM1 GROUP 5',
    'Lecturer': 'Dr. Burhan UI Islam Khan',
    'Demostrator': 'Ng Zhi Yang',
    'Lab Time': '5pm - 7pm',
}

## Students list
The tutorial's student list needs to be processed once.

In [4]:
tutorial_student_list = "courseid_151_participants.xlsx"
processed_data = pd.read_excel(tutorial_student_list) \
                    .drop(['Surname', 'Email address'], axis=1) \
                    .rename(columns={'ID number': 'Student ID'}) \
                    .dropna() \
                    .sort_values(by='Student ID')
processed_data.to_csv("WIX1002_2324_Student-List.csv", index=None, header=True)

## Converting Excel to CSV

For this entire notebook, you need to manually paste the excel locations file by file into here. If you have multiple files you just paste it one by one.
If you know how to put all files into a directory and use just one loop no matter the number of files, do send in a PR.

In [5]:
students = pd.read_csv("WIX1002_2324_Student-List.csv")

In [6]:
def extract_date(date_time):
    date_pattern = r'\d{1,3}-\d{1,3}-\d{4}'
    return re.search(date_pattern, date_time).group()

In [7]:
date_string = []
file_string = []
files = os.listdir(folder_path)
filtered_files = [file for file in files if file.startswith(file_prefix)]

for file_name in filtered_files:
    file = os.path.join(folder_path, file_name)
    excel_file = pd.read_excel(file)
    date = extract_date(excel_file.iloc[1,1])
    date_string.append(date)
    csv_file_path = os.path.join(folder_path, f"Attendance_{date}.csv")
    excel_file.to_csv(csv_file_path, index=None, header=True)

## CSV overview

In [8]:
def process_attendance_data(csv_file):
    df = pd.read_csv(csv_file)
    df = df.iloc[4:, :4]
    df = df.rename(columns={'Course': 'Student ID',
                           'WIX1002 FUNDAMENTALS OF PROGRAMMING': 'Name',
                           'Unnamed: 2': 'Email address',
                           'Unnamed: 3': 'Status'}).sort_values(by='Student ID')
    return df

In [10]:
for date, file_name in zip(date_string, filtered_files):
    csv_file_path = os.path.join(folder_path, f"Attendance_{date}.csv")
    df = process_attendance_data(csv_file_path)
    df = df[df['Student ID'].isin(students['Student ID'])]
    students[date] = df['Status'].tolist()

students['No'] = range(1, len(df) + 1)
students = students[['No'] + [col for col in students if col != 'No']]

## Working with workbook

In [11]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

In [19]:
wb = Workbook()
ws = wb.active

In [20]:
# Resize columns
column_widths = [12.78, 29.11, 10.78, 11, 11, 11, 11, 11]  # Set the desired column widths
    
for column_index, width in enumerate(column_widths, start=1):
    column_letter = get_column_letter(column_index)
    ws.column_dimensions[column_letter].width = width

In [21]:
for i, (key, value) in enumerate(info.items(), start=1):
    cell = ws.cell(row=i, column=1, value=key)
    cell.alignment = Alignment(horizontal='right')
    cell = ws.cell(row=i, column=2, value=value)

In [22]:
starting_row = 7

for i, row in enumerate(dataframe_to_rows(students, index=False, header=True), start=starting_row):
    for j, value in enumerate(row, start=1):
        cell = ws.cell(row=i, column=j, value=value)

In [23]:
wb.save(f'Final_Attendance_{month}.xlsx')